In [14]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/databhavya/AI/AI gen/Various-AI-portraits-generated-by-Fotor.jpg
/kaggle/input/databhavya/AI/AI gen/images77.jpg
/kaggle/input/databhavya/AI/AI gen/1000_F_563719058_JXnzcPV4GRpWqmF5sqnqmbJ7ow3ca3DS.jpg
/kaggle/input/databhavya/AI/AI gen/MidJourney-content-policy-1024x576.jpg
/kaggle/input/databhavya/AI/AI gen/an-ai-jungle-landscape-made-by-ai-landscape-generator.jpg
/kaggle/input/databhavya/AI/AI gen/bDSF6saJUuPLsEu9i3ud--1--gvs7i.jpg
/kaggle/input/databhavya/AI/AI gen/ai-last-selfie-taken-earth-comp.jpg
/kaggle/input/databhavya/AI/AI gen/google-dream-starry-night.jpg
/kaggle/input/databhavya/AI/AI gen/panoramic-abstract-mountain-scenery-in-flat-style-natural-wallpaper-ai-generated-image-photo.jpg
/kaggle/input/databhavya/AI/AI gen/ai-generated-8067660_1280.jpg
/kaggle/input/databhavya/AI/AI gen/14PMNtflOIXcw4g9HTntybg.png
/kaggle/input/databhavya/AI/AI gen/1E9WJo1jFlzdGVuFhb4BdQw.png
/kaggle/input/databhavya/AI/AI gen/GoogleAI-theAIgang.jpg
/kaggle/input/databhavya/AI/AI

In [12]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from skimage.feature import local_binary_pattern
from joblib import Parallel, delayed
import os

# Parameters
img_size = (128, 128)
batch_size = 32
epochs = 16
n_jobs = -1  # Use all available cores

# Directories
data_dir = '/kaggle/input/databhavya/AI'  # The parent directory containing 'real' and 'fake' folders

# Image Data Generators
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training',
    shuffle=True
)

val_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation',
    shuffle=False
)

# Load Pre-trained MobileNetV2 Model
base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=img_size + (3,)
)

# Freeze the base model layers
base_model.trainable = False

# Add custom layers
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

# Create MobileNetV2-based model
mobilenet_model = tf.keras.Model(inputs=base_model.input, outputs=outputs)

# Compile the model
mobilenet_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
mobilenet_model.fit(train_generator, validation_data=val_generator, epochs=epochs)

# Save the MobileNetV2 model predictions
train_preds = mobilenet_model.predict(train_generator)
val_preds = mobilenet_model.predict(val_generator)

Found 777 images belonging to 2 classes.
Found 193 images belonging to 2 classes.
Epoch 1/16


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 5/25 ━━━━━━━━━━━━━━━━━━━━ 14s 744ms/step - accuracy: 0.6357 - loss: 0.9252

/opt/conda/lib/python3.10/site-packages/PIL/Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


25/25 ━━━━━━━━━━━━━━━━━━━━ 39s 964ms/step - accuracy: 0.6025 - loss: 0.9361 - val_accuracy: 0.7098 - val_loss: 0.5240
Epoch 2/16
25/25 ━━━━━━━━━━━━━━━━━━━━ 23s 692ms/step - accuracy: 0.8118 - loss: 0.4361 - val_accuracy: 0.7617 - val_loss: 0.5018
Epoch 3/16
25/25 ━━━━━━━━━━━━━━━━━━━━ 23s 716ms/step - accuracy: 0.8460 - loss: 0.3465 - val_accuracy: 0.8031 - val_loss: 0.4739
Epoch 4/16
25/25 ━━━━━━━━━━━━━━━━━━━━ 41s 716ms/step - accuracy: 0.8507 - loss: 0.3187 - val_accuracy: 0.7824 - val_loss: 0.4392
Epoch 5/16
25/25 ━━━━━━━━━━━━━━━━━━━━ 23s 766ms/step - accuracy: 0.9351 - loss: 0.2163 - val_accuracy: 0.8342 - val_loss: 0.4324
Epoch 6/16
25/25 ━━━━━━━━━━━━━━━━━━━━ 23s 712ms/step - accuracy: 0.9582 - loss: 0.1643 - val_accuracy: 0.8394 - val_loss: 0.4156
Epoch 7/16
25/25 ━━━━━━━━━━━━━━━━━━━━ 23s 674ms/step - accuracy: 0.9764 - loss: 0.1308 - val_accuracy: 0.8187 - val_loss: 0.4165
Epoch 8/16
25/25 ━━━━━━━━━━━━━━━━━━━━ 24s 700ms/step - accuracy: 0.9600 - loss: 0.1226 - val_accuracy: 0.829

In [36]:
def extract_color_histogram(image_path, bins=(8, 8, 8)):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (128, 128))  # Resize to match model input
    hist = cv2.calcHist([img], [0, 1, 2], None, bins, [0, 256, 0, 256, 0, 256])
    hist = cv2.normalize(hist, hist).flatten()  # Normalize and flatten the histogram
    return hist

# Parallel processing to extract color histogram features
train_color_features = Parallel(n_jobs=-1)(delayed(extract_color_histogram)(img_path) for img_path in train_generator.filepaths)
val_color_features = Parallel(n_jobs=-1)(delayed(extract_color_histogram)(img_path) for img_path in val_generator.filepaths)

# Convert to arrays
train_color_features = np.array(train_color_features)
val_color_features = np.array(val_color_features)


libpng warning: iCCP: profile 'ICC Profile': 'CMYK': invalid ICC profile color space


In [39]:
def extract_gabor_features(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (128, 128))  # Resize to match model input
    gabor_features = []
    
    for theta in range(4):  # 4 orientations
        theta = theta / 4. * np.pi
        kernel = cv2.getGaborKernel((21, 21), 8.0, theta, 10.0, 0.5, 0, ktype=cv2.CV_32F)
        filtered = cv2.filter2D(img, cv2.CV_8UC3, kernel)
        gabor_features.append(cv2.calcHist([filtered], [0], None, [256], [0, 256]).flatten())

    return np.concatenate(gabor_features)

# Parallel processing to extract Gabor features
train_gabor_features = Parallel(n_jobs=-1)(delayed(extract_gabor_features)(img_path) for img_path in train_generator.filepaths)
val_gabor_features = Parallel(n_jobs=-1)(delayed(extract_gabor_features)(img_path) for img_path in val_generator.filepaths)

# Convert to arrays
train_gabor_features = np.array(train_gabor_features)
val_gabor_features = np.array(val_gabor_features)


libpng warning: iCCP: profile 'ICC Profile': 'CMYK': invalid ICC profile color space


In [40]:
# Combine MobileNet predictions, Color Histogram, and Gabor features
train_combined = np.hstack((train_preds, train_color_features, train_gabor_features))
val_combined = np.hstack((val_preds, val_color_features, val_gabor_features))


In [44]:
from sklearn.model_selection import GridSearchCV

# Create a pipeline if you want to combine steps
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline

# Define the model
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

# Set up the parameter grid
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2]
}

# Perform grid search
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, n_jobs=-1, scoring='accuracy')
grid_search.fit(train_combined, train_generator.classes)

# Get the best parameters
print("Best parameters found: ", grid_search.best_params_)
best_xgb_model = grid_search.best_estimator_

# Evaluate the model with the best parameters
val_preds_best_xgb = best_xgb_model.predict(val_combined)
print(classification_report(val_generator.classes, val_preds_best_xgb))


/opt/conda/lib/python3.10/site-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


Best parameters found:  {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100}
              precision    recall  f1-score   support

           0       0.73      0.78      0.75       107
           1       0.70      0.65      0.67        86

    accuracy                           0.72       193
   macro avg       0.72      0.71      0.71       193
weighted avg       0.72      0.72      0.72       193



198551472/198551472 ━━━━━━━━━━━━━━━━━━━━ 9s 0us/step
